# Preprocessing Dataset Image

La fase di preprocessing, come visto nei casi precedenti, è un passo fondamentale per la manipolazione dei dati che avverà negli step successivi. Il dataset che ci è stato fornito per poter avviare la classificazione attraverso una  _Convolutional Deep Neural Network_ è composto essenzialmente da due file principali per per ogni caso di studio: 
1. Il file GeoTiff contenente l'immagine satellitare SAR e le informazione di georeferenziazione. 
2. La TagMask di ogni GeoTiff calcolata per ogni caso di studio.

L'input di una rete Convoluzionale deve rispettare determinate caratteristiche dettate da proprietà intrinseche della stessa. Un aspetto cruciale sono le dimensioni delle immagini in input, un dataset per una rete convoluzionale deve essere formato da immagini quadrate con le medesime dimensioni. 
Le immagini che compongono il nostro dataset non rispettano questo vincolo per i seguenti motivi:
1. il dataset è costituito da CaseStudy, ogni CaseStudy è una porzione di immagine rilevata dal satellite, da questo momento chiameremo _"quarti"_.
2. ogni _quarto_ ha dimensione variabile in numero di pixel e al suo interno ha una o più macchie di petrolio.
3. le macchie di petrolio si distinguono principalmente per aspetti geometrici tra i quali forma e area.

Il primo passo fondamentale di questa fase di preprocessing è stata quella dell'estrazione delle macchie di petrolio dai singoli _quarti_.

Analizziamo ora lo script utilizzato, partiamo con le costanti che riguardano:
1. *path\_geotiff* nome della cartella dove sono memorizzati i file geotiff
2. *path\_mask* nome cartella dove sono memorizzati le TagMask 
3. *out\_dir* path del salvataggio
4. _class2_ _class3_ e _class4_ sono i nomi delle tre classi inerenti il grado di pericolosità delle macchie di petrolio

In [15]:
clear all % pulpizia del workspace

patch_geotiff = '/GeoTIFF/';
patch_mask = '/OilSpillDetection/';
patch_dataset = './dataset/';
out_dir = './crop/';
class2 = num2str(2);
class3 = num2str(3);
class4 = num2str(4);

own = 32; % dimensione delle immagini di output
img_zero= zeros(own, own); % immagine contenente tutti zero con dimesione (own x own)

tot_crop = 0; % contatore per il numero di macchine trovate 

BIN = 1; % costante che definisce se utilizzare la TagMask o il GeoTiff per generare i crop

Fatto questo lo script può proseguire verificando se nel percoso di salvataggio è presente una cartella precedetemente generata; se questo si verifica verrà eliminata per far spazio alla nuova generazione.

In [16]:
if exist(out_dir, 'dir')
    rmdir(out_dir, 's');
end
% creazione della struttura per il salvataggio delle immagini in output
mkdir(out_dir);
mkdir(strcat(out_dir,class2));
mkdir(strcat(out_dir,class3));
mkdir(strcat(out_dir,class4));

Preparato il path per il savataggio delle OilSpill lo script può analizzare la struttura del dataset di input. L'analisi del dataset parte dalla directory dove è contenuto lo script _./_ ed estrapola l'elenco delle cartelle che rapprensentano i vari _quarti_ precedentemente citati. In *list\_dir* avremo ora i path da analizzare.

In [17]:
list_dir = dir(patch_dataset);
list_dir = extractfield(list_dir([list_dir.isdir]), 'name');
list_dir = list_dir(3:length(list_dir));

Di seguito lo script avvia due cicli _for_ incampsulati. Il ciclo _for_ esterno carica per ogni _quarto_ individuato nel dataset, il _geotiff_ e la _TagMask_ che subiranno un primo passaggio di pulizia da eventuali macchine molto piccole; _MinAreaRem_.
Terminata questa fase che possiamo considerare globale sul _quarto_ in esame lo script passa al ciclo _for_ interno che analizza ogni singola macchina identificata. 
Per ottenere immagini consone all'utilizzo in una _Convolutional Neural Network_ lo script per ogni macchina identificata esegue i seguenti fondamentali passaggi:
1. crop della macchia dal _quarto_ in esame 
2. resize del crop mantenendo le stesse proporzioni portando quindi il lato maggiore alla dimensione desiderata
3. aggiunta del padding per il lato di dimensione inferiore per rendere l'output quadrato 
al termine del lo script esegue un'ulteriore verifica prima di poter salvare l'output, cerca se all'interno dell'immagine ci sono aree isolate generate della precedente fase di ritaglio. Se questo si verifica vengono eliminate garantendo che all'interno dell'output ci sia solo una macchina. 
L'immagine quindi viene salvata nel percorso corrispondente al tag precedentemente letto.
Al termine dello script avremo il numero totale di macchie trovate nel dataset. 

In [18]:
%% AVVIO CICLO FOR ESTERNO PER IL NUMERO DI "QUARTI" INDIVIDUATI NEL DATASET

for i=1:length(list_dir)

    % loading del "quarto" selezionato dalla cartella del dataset
    geo_file = ls(strcat(patch_dataset,char(list_dir(i)),patch_geotiff));
    geo_file = strcat(patch_dataset,char(list_dir(i)),patch_geotiff,'/',geo_file);
    geotiff = geotiffread(geo_file);
    
    mask_file = dir(fullfile(patch_dataset,strcat(char(list_dir(i)),patch_mask),'*tif'));
    mask_file = extractfield(mask_file, 'name');
    mask_file = strcat(patch_dataset,char(list_dir(i)),patch_mask,'/',mask_file);
    TagMask = imread(char(mask_file));
    
    % avvio fase di pulizia e idetificazione delle macchie di petrolio
    
    MinAreaRem=3*150; % definizione area minima della macchia
    
    Isent = geotiff(:,:,1); % primo livello radiometrico del geotiff
    I=Isent;
    Inan=find(isnan(I));
    I(Inan)=0;
    
    ind_tag=find(TagMask>=2); % dalla TagMask copiamo le area con indice superiore o pari a 2 
    SlickMask=zeros(size(TagMask));
    Mask_tag = zeros(size(TagMask));
    SlickMask(ind_tag)=1;
    Mask_tag(ind_tag) = TagMask(ind_tag);
    
    Mask=bwareaopen(SlickMask,MinAreaRem); % eliminazione delle aree inferiori a MinAreaRem
    
    [II_Lab nL]= bwlabel(Mask); 
    
    STATS = regionprops(II_Lab,I,'BoundingBox'); % regioprops restituisce in STATS il numero di macchie 
                                                 % con le coordinate del BoundingBox
    
    fprintf('%s Numero Macchie : %d \n',string(list_dir(i)), length(STATS));
    
    tot_crop = tot_crop + length(STATS);

    %% AVVIO CICLO FOR INTERNO PER OGNI MACCHINA INDIVIDUATA NEL QUARTO 
    
    for j=1:length(STATS)
        
        % carico il crop dal geotiff 
        tiff_crop = imcrop(I,STATS(j).BoundingBox);
        % carico il crop dalla TagMask
        mask_crop = imcrop(TagMask,STATS(j).BoundingBox);
        
        % identifico il tag di quella macchina 
        tag = max(max(mask_crop));
        
        if BIN == 1 
            idx = find(mask_crop > 1);
            geo_crop = zeros(size(mask_crop));
            geo_crop(idx) = 1;
        else 
            geo_crop = tiff_crop;
        end
        
        [m,n] = size(geo_crop); % dimensione del crop
        K_pad = img_zero;
        
        % avvio fase di resize del crop in base al lato maggiore. viene aggiunto il padding al lato inferiore. 
        if m > n
            a = imresize(geo_crop,[own NaN]);
            k = abs(own-size(a,2));
            K_pad(1:own,ceil(k/2)+1:ceil(k/2)+size(a,2)) = a(:,:);
        else
            a = imresize(geo_crop,[NaN own]);
            k = abs(own-size(a,1));
            K_pad(ceil(k/2)+1: ceil(k/2)+size(a,1),1:own) = a(:,:);
        end
       
        % K_pad=bwareaopen(K_pad,5);
        [Lab nL2]= bwlabel(K_pad);
        
        % ulteriore pulizia del crop appena genrato da eventuali macchine non intere dovute al taglio dal "quarto"
        aree = regionprops(K_pad, 'Area', 'BoundingBox');
        
        if (nL2 > 1 )
            for k=2:length(aree)
                BB = aree(k).BoundingBox;
                x1 = ceil(BB(1));
                x2 = round(x1 + BB(3));
                y1 = ceil(BB(2));
                y2 = round(y1 + BB(4));
                K_pad(y1:y2, x1:x2) = 0;
            end
        end

        out_file = strcat(out_dir,num2str(tag),'/crop_',num2str(tag),'_',num2str(i),num2str(j),'.png');
        imwrite(K_pad,out_file);
    end
end

fprintf('Totale crop : %d \n', tot_crop); % numero totale delle macchine trovate nel dataset 

CaseStudy_01 Numero Macchie : 110 
CaseStudy_02 Numero Macchie : 199 
CaseStudy_03 Numero Macchie : 89 
CaseStudy_04 Numero Macchie : 132 
Totale crop : 530 
